# Testing StarDist network for instance segmentation

In [1]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib
matplotlib.rcParams["image.interpolation"] = 'none'
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tqdm import tqdm
import skimage as sk
from tifffile import imread
from csbdeep.utils import Path, normalize
import napari
import os
from stardist import fill_label_holes, random_label_cmap, calculate_extents, gputools_available
from stardist.matching import matching, matching_dataset
from stardist.models import Config2D, StarDist2D, StarDistData2D

np.random.seed(42)
lbl_cmap = random_label_cmap()

Use the following 2 cells to convert pngs of masks exported from QuPath to Tiffs, if needed

In [ ]:
PNGs = sorted(glob('E:/Grainger_Lab/Amber/OIC-74_Zebrafish_RBC_Classification/tiles/**/instTiles/*.png',recursive=True))

In [ ]:
#Converted all pngs to tiffs
for y in PNGs:
    path = os.path.dirname(y)
    name = os.path.basename(y)
    img = sk.io.imread(y)
    sk.io.imsave(os.path.join(path,name[:-4]+'.tif'),img,check_contrast=False)

Read in images

In [56]:
X = sorted(glob('E:/Grainger_Lab/Amber/OIC-74_Zebrafish_RBC_Classification/tiles/2025_01_28__0346-Scene-1-ScanRegion0/imgTiles/*.tif',recursive=True))
Y = sorted(glob('E:/Grainger_Lab/Amber/OIC-74_Zebrafish_RBC_Classification/tiles/2025_01_28__0346-Scene-1-ScanRegion0/instTiles/*.tif',recursive=True))
assert all(Path(x).name==Path(y).name for x,y in zip(X,Y)) #added indexing here because the file type is different between the images and the masks, using indexing to match the name of the file without file type

In [57]:
X = list(map(imread,X))
Y = list(map(imread,Y))

In [59]:
n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]

In [60]:
# Check number of channels
print(n_channel)

3


Normalize the images, fill possible holes in labels then split into train and validate groups

In [61]:
axis_norm = (0,1)   # normalize channels independently
# axis_norm = (0,1,2) # normalize channels jointly
if n_channel > 1:
    print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))
    sys.stdout.flush()

X = [normalize(x,1,99.8,axis=axis_norm) for x in tqdm(X)]
Y = [fill_label_holes(y) for y in tqdm(Y)]

Normalizing image channels independently.


100%|████████████████████████████████████████████████████████████████████████████████| 156/156 [00:04<00:00, 34.87it/s]


In [62]:
assert len(X) > 1, "not enough training data"
rng = np.random.RandomState(42)
ind = rng.permutation(len(X))
n_val = max(1, int(round(0.15 * len(ind))))
ind_train, ind_val = ind[:-n_val], ind[-n_val:]
X_val, Y_val = [X[i] for i in ind_val]  , [Y[i] for i in ind_val]
X_trn, Y_trn = [X[i] for i in ind_train], [Y[i] for i in ind_train] 
print('number of images: %3d' % len(X))
print('- training:       %3d' % len(X_trn))
print('- validation:     %3d' % len(X_val))

number of images: 156
- training:       133
- validation:      23


Set up hyperparameters for StarDist Model

In [63]:
# 32 is a good default choice (see 1_data.ipynb)
n_rays = 32

# Use OpenCL-based computations for data generator during training (requires 'gputools')
use_gpu = True and gputools_available()

# Predict on subsampled grid for increased efficiency and larger field of view
grid = (4,4)

conf = Config2D (
    n_rays       = n_rays,
    grid         = grid,
    use_gpu      = use_gpu,
    n_channel_in = n_channel,
    train_patch_size = (512,512),
    train_steps_per_epoch = 100,
    train_epochs = 400,
)
#print(conf)
vars(conf)

{'n_dim': 2,
 'axes': 'YXC',
 'n_channel_in': 3,
 'n_channel_out': 33,
 'train_checkpoint': 'weights_best.h5',
 'train_checkpoint_last': 'weights_last.h5',
 'train_checkpoint_epoch': 'weights_now.h5',
 'n_rays': 32,
 'grid': (4, 4),
 'backbone': 'unet',
 'n_classes': None,
 'unet_n_depth': 3,
 'unet_kernel_size': (3, 3),
 'unet_n_filter_base': 32,
 'unet_n_conv_per_depth': 2,
 'unet_pool': (2, 2),
 'unet_activation': 'relu',
 'unet_last_activation': 'relu',
 'unet_batch_norm': False,
 'unet_dropout': 0.0,
 'unet_prefix': '',
 'net_conv_after_unet': 128,
 'net_input_shape': (None, None, 3),
 'net_mask_shape': (None, None, 1),
 'train_shape_completion': False,
 'train_completion_crop': 32,
 'train_patch_size': (512, 512),
 'train_background_reg': 0.0001,
 'train_foreground_only': 0.9,
 'train_sample_cache': True,
 'train_dist_loss': 'mae',
 'train_loss_weights': (1, 0.2),
 'train_class_weights': (1, 1),
 'train_epochs': 400,
 'train_steps_per_epoch': 100,
 'train_learning_rate': 0.0003,


In [52]:
model = StarDist2D(conf, name='BloodCells', basedir='models')

Using default values: prob_thresh=0.5, nms_thresh=0.4.


Make sure that the field of view for the network is larger than the objects

In [64]:
median_size = calculate_extents(list(Y), np.median)
fov = np.array(model._axes_tile_overlap('YX'))
print(f"median object size:      {median_size}")
print(f"network field of view :  {fov}")
# if any(median_size > fov):
#     print("WARNING: median object size larger than field of view of the neural network.")

median object size:      [91. 91.]
network field of view :  [189 189]


Define the augementations to use

In [65]:
def random_fliprot(img, mask): 
    assert img.ndim >= mask.ndim
    axes = tuple(range(mask.ndim))
    perm = tuple(np.random.permutation(axes))
    img = img.transpose(perm + tuple(range(mask.ndim, img.ndim))) 
    mask = mask.transpose(perm) 
    for ax in axes: 
        if np.random.rand() > 0.5:
            img = np.flip(img, axis=ax)
            mask = np.flip(mask, axis=ax)
    return img, mask 

def random_intensity_change(img):
    img = img*np.random.uniform(0.6,2) + np.random.uniform(-0.2,0.2)
    return img


def augmenter(x, y):
    """Augmentation of a single input/label image pair.
    x is an input image
    y is the corresponding ground-truth label image
    """
    x, y = random_fliprot(x, y)
    x = random_intensity_change(x)
    # add some gaussian noise
    sig = 0.02*np.random.uniform(0,1)
    x = x + sig*np.random.normal(0,1,x.shape)
    return x, y

Train the model

Use `tensorboard --logdir=.` in the command line in the same parent directory as the models (with StarDist env active) to watch live read out of training (could also put in the directory of the parent folder instead of cd to parent directory)

In [66]:
model.train(X_trn, Y_trn, validation_data=(X_val,Y_val), augmenter=augmenter)

Epoch 1/400
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 628ms/step - dist_dist_iou_metric: 0.0721 - dist_loss: 33.3567 - dist_relevant_mae: 33.3558 - dist_relevant_mse: 1581.2866 - loss: 7.2958 - prob_kld: 0.4284 - prob_loss: 0.6245WARNING:tensorflow:5 out of the last 9 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x00000178904ED1C0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


100/100 ━━━━━━━━━━━━━━━━━━━━ 72s 655ms/step - dist_dist_iou_metric: 0.0731 - dist_loss: 33.2973 - dist_relevant_mae: 33.2965 - dist_relevant_mse: 1577.1763 - loss: 7.2838 - prob_kld: 0.4282 - prob_loss: 0.6243 - val_dist_dist_iou_metric: 0.1664 - val_dist_loss: 22.7566 - val_dist_relevant_mae: 22.7244 - val_dist_relevant_mse: 840.6803 - val_loss: 4.8862 - val_prob_kld: 0.1570 - val_prob_loss: 0.3435 - learning_rate: 3.0000e-04
Epoch 2/400
100/100 ━━━━━━━━━━━━━━━━━━━━ 61s 618ms/step - dist_dist_iou_metric: 0.3404 - dist_loss: 18.9244 - dist_relevant_mae: 18.9213 - dist_relevant_mse: 589.6682 - loss: 4.1632 - prob_kld: 0.1697 - prob_loss: 0.3783 - val_dist_dist_iou_metric: 0.3610 - val_dist_loss: 16.2768 - val_dist_relevant_mae: 16.2552 - val_dist_relevant_mse: 474.5856 - val_loss: 3.8963 - val_prob_kld: 0.4597 - val_prob_loss: 0.6462 - learning_rate: 3.0000e-04
Epoch 3/400
100/100 ━━━━━━━━━━━━━━━━━━━━ 60s 605ms/step - dist_dist_iou_metric: 0.4524 - dist_loss: 14.3511 - dist_relevant_mae

KeyboardInterrupt: 

In [ ]:
model.optimize_thresholds(X_val, Y_val)